In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from geobases.GeoBases import GeoBase

# Make the graphs a bit prettier, and bigger
pd.set_option('display.mpl_style', 'default')

# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

geo_a = GeoBase(data='airports', verbose=False)

print "Let's start the challenge! "

ImportError: No module named geobases.GeoBases

In [4]:
bookings = pd.read_csv('data/bookings.csv.bz2', sep='^', compression='bz2', usecols=['arr_port','pax'], nrows=100000)
print "The number of rows of bookings file is %s" % len(bookings)

The number of rows of bookings file is 100000


In [6]:
searches = pd.read_csv('data/searches.csv.bz2', sep='^', compression='bz2', usecols=['Destination','Origin'], nrows=100000)
print "The number of rows of searches file is %s" % len(searches)

The number of rows of searches file is 100000


In [7]:
best10airports = bookings.groupby('arr_port').aggregate(sum).sort_index(by='pax', ascending=False).head(10)
print best10airports

           pax
arr_port      
LHR       1006
MCO        861
JFK        795
LAX        761
BKK        747
LAS        732
SFO        705
ORD        686
CDG        676
DXB        587
